In [1]:

from fastai.vision.all import *
pip 

set_seed(42)

path_ori = Path('/kaggle/input/clothing-dataset-full/images_original')
path_comp = Path('/kaggle/input/extracting-attributes-from-fashion-images-2/train')


In [2]:
import pandas as pd

In [3]:
img_csv = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/train.csv')

In [4]:
train_path = '/kaggle/input/extracting-attributes-from-fashion-images-2/train'
train_files = get_image_files(train_path)

test_path = '/kaggle/input/extracting-attributes-from-fashion-images-2/test'
test_files = get_image_files(test_path).sorted()


train_df = pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/train.csv')
train_df.head()

,file_name,label
0,Image_train_00001.jpg,0
1,Image_train_00002.jpg,1
2,Image_train_00003.jpg,0
3,Image_train_00004.jpg,0
4,Image_train_00005.jpg,0


In [5]:
train_df.dtypes

file_name    object
label         int64
dtype: object

In [6]:
blocks = (ImageBlock, CategoryBlock)
blocks

(<function fastai.vision.data.ImageBlock(cls: 'PILBase' = <class 'fastai.vision.core.PILImage'>)>,
 <function fastai.data.block.CategoryBlock(vocab: 'list | pd.Series' = None, sort: 'bool' = True, add_na: 'bool' = False)>)

In [7]:
get_x = ColReader("file_name", pref=(path_comp))
get_y = ColReader("label")
get_x
get_y

ColReader -- {'cols': 'label', 'pref': '', 'suff': '', 'label_delim': None}:
encodes: decodes: 

In [8]:
from PIL import Image
image = Image.open('/kaggle/input/extracting-attributes-from-fashion-images-2/train/Image_train_00003.jpg')
width, height = image.size
width

400

In [9]:
height

533

In [10]:
from PIL import Image
image = Image.open('/kaggle/input/extracting-attributes-from-fashion-images-2/test/Image_test_00003.jpg')
width, height = image.size
width

400

In [11]:
height

533

In [29]:
tfms = aug_transforms(do_flip=True, flip_vert=True, max_rotate=0.10, max_zoom=1.3, max_warp=0.0)


In [35]:
dblock = DataBlock(blocks=blocks, 
                    get_x=get_x,
                    get_y=get_y,
                   item_tfms=Resize(480,method='squish'),
                   batch_tfms=tfms,splitter=RandomSplitter(valid_pct=0.2, seed=42))
dblock


In [14]:
aug_transforms?

Signature:
aug_transforms(
    mult: 'float' = 1.0,
    do_flip: 'bool' = True,
    flip_vert: 'bool' = False,
    max_rotate: 'float' = 10.0,
    min_zoom: 'float' = 1.0,
    max_zoom: 'float' = 1.1,
    max_lighting: 'float' = 0.2,
    max_warp: 'float' = 0.2,
    p_affine: 'float' = 0.75,
    p_lighting: 'float' = 0.75,
    xtra_tfms: 'list' = None,
    size: 'int | tuple' = None,
    mode: 'str' = 'bilinear',
    pad_mode='reflection',
    align_corners=True,
    batch=False,
    min_scale=1.0,
)
Docstring: Utility func to easily create a list of flip, rotate, zoom, warp, lighting transforms.
File:      /opt/conda/lib/python3.7/site-packages/fastai/vision/augment.py
Type:      function


In [31]:
dls = dblock.dataloaders(img_csv)
dls

In [ ]:
dls.show_batch()

In [36]:
learn = vision_learner(dls, resnet34, metrics=accuracy,pool=nn.AdaptiveAvgPool2d)

learn


In [ ]:
lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
learn.fit_one_cycle(3,lrs.valley)

In [ ]:
learn.unfreeze()
lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
learn.fit_one_cycle(12,slice(lrs.minimum,lrs.slide))

In [ ]:
learn.fine_tune(10,0.001737800776027143)

In [ ]:
probs, target = learn.get_preds(dl=dls.valid)
error_rate(probs, target)

In [ ]:
probs, target = learn.tta(dl=dls.valid)
error_rate(probs, target)

In [ ]:
probs, _ = learn.tta(dl=dls.test_dl(test_files))

In [ ]:
preds = probs.argmax(dim=1)


In [ ]:
dls.vocab[preds]


In [ ]:
sample_submission=pd.read_csv('/kaggle/input/extracting-attributes-from-fashion-images-2/sample_submission.csv')

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.label = dls.vocab[preds]


In [ ]:
import pandas

In [ ]:
import numpy

In [ ]:
sample_submission.to_csv('112121.csv',index=False)


In [ ]:
FileLink("111.csv")